In [1]:
import wisps 
import splat
import numpy as np
import numba
import matplotlib.pyplot as plt

In [5]:
import splat.simulate as spsim

In [7]:
density_function=spsim.galactic_density_juric

In [10]:
import numpy as np
import scipy.integrate as si
import numba
from numba import cfunc,carray
from numba.types import intc, CPointer, float64
from scipy import LowLevelCallable

def jit_integrand_function(integrand_function):
    jitted_function = numba.jit(integrand_function, nopython=True)
    @cfunc(float64(intc, CPointer(float64)))
    def wrapped(n, xx):
        values = carray(xx,n)
        return jitted_function(values)
    return LowLevelCallable(wrapped.ctypes)

@jit_integrand_function
def integrand(args):
    return -np.exp(args.prod())

#Two variable, two parameter example
parms = np.array([2,3])
print (si.nquad(integrand,[[0,1],[0,1]],parms))

#Three variable, three parameter example
parms2 = np.array([1,2,3])
print (si.nquad(integrand,[[0,1],[0,1],[0,1]],parms2))

(-13.93679783471827, 7.365796628137861e-13)
(-4.114488880754228, 7.2862413532559e-13)


In [11]:
%timeit si.nquad(integrand,[[0,1],[0,1]],parms)

355 µs ± 20.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
density_function(r)

Signature:
density_function(
    rc,
    zc,
    rho0=<Quantity 1. 1 / pc3>,
    report='total',
    center='sun',
    unit=Unit("pc"),
    **kwargs,
)
Docstring:
:Purpose: 

    Returns the local galactic star density at galactic radial (r) and vertical (z) coordinates relative to an assumed "local" density. 
    for the Galaxy model of `Juric et al. (2008, ApJ, 673, 864) <http://adsabs.harvard.edu/abs/2008ApJ...673..864J>`_
    Coordinates are sun-centered unless otherwise specified

:Required Inputs:

    :param rc: single or array of floating points of galactic radial coordinates, assumed to be in units of pc
    :param zc: single or array of floating points of galactic vertical coordinates, assumed to be in units of pc

:Optional Inputs:

    :param: rho0 = 1./pc^3: local number density
    :param: center = 'sun': assumed center point, by default 'sun' but could also be 'galaxy'
    :param: report = 'total: what density to report:

        * 'total': (default) report the total gal